In [1]:
import sys,os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.data.cancer_sim.tumor_dataset_pipeline import TumorGrowthDatasetPipeline
import omegaconf
import numpy as np
from hydra.utils import instantiate, get_original_cwd
config_path = '../config/dataset/tumor_debug.yaml'


c:\Users\mhr_k\miniconda3\envs\R-learner\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
args = omegaconf.OmegaConf.load(config_path)
args.dataset.seed = 2026
data_pipeline = instantiate(args.dataset)

  0%|          | 0/100 [00:00<?, ?it/s]c:\Users\mhr_k\OneDrive\Documents\Thesis\Neural-R-Learner\src\data\cancer_sim\simulation.py:324: RuntimeWarning: overflow encountered in exp
  if recovery_rvs[i, t] < np.exp(-cancer_volume[i, t] * TUMOUR_CELL_DENSITY):
100%|██████████| 100/100 [00:00<00:00, 2855.37it/s]


In [3]:
test_data = data_pipeline.test_data

In [6]:
T_intv = np.ones((args.dataset.n_periods, 2))
T_base = np.zeros((args.dataset.n_periods, 2))
y_intv = data_pipeline._simulate_counterfactuals('test', T_intv)
y_base = data_pipeline._simulate_counterfactuals('test', T_base)
te_1 = y_intv - y_base
te_1.shape

treatment_seq: [[1. 1.]
 [1. 1.]
 [1. 1.]]
treatment_seq: [[0. 0.]
 [0. 0.]
 [0. 0.]]


c:\Users\mhr_k\OneDrive\Documents\Thesis\Neural-R-Learner\src\data\cancer_sim\simulation.py:444: RuntimeWarning: overflow encountered in exp
  recovery_idx = np.exp(-ctf_volume[:, t + k] * TUMOUR_CELL_DENSITY) > 0.95


(15, 28)

In [4]:
test_index = data_pipeline.index['test']
ind_de = data_pipeline.compute_individual_true_dynamic_effects(data_pipeline.data['cancer_volume'][test_index])
te_2 = (ind_de * np.expand_dims(np.expand_dims((T_intv - T_base), axis = 0), axis = 0)).sum(axis = (2, 3))
te_2.shape

(15, 28)

In [7]:
((te_1 - te_2)**2 / te_1**2).mean(axis = 0)

array([0.00399983, 0.00399984, 0.00399983, 0.00399983, 0.00399983,
       0.00399983, 0.00399983, 0.00399983, 0.00399983, 0.00399983,
       0.00399983, 0.00399983, 0.00399983, 0.00399983, 0.00399983,
       0.00399983, 0.00399983, 0.00399983, 0.00399983, 0.00399983,
       0.00399983, 0.00399983, 0.00399983, 0.00399983, 0.00399983,
       0.00399983, 0.00399983, 0.00399983])